In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Demographic Data Processing


In [2]:
def remove_minus(x):
    if (x == "-"):
        return "NaN"
    else:
        return x.replace("-", "").replace("+", "").replace(",", "")

In [15]:
#this data is for 11 variables, all census tracts in California, using ACS 2009 5YR estimates 

demog_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/demographic_data/'
#get percentages of each race

race = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B03002_race_clean.csv')
race["Percent White alone"] = race["Estimate; Not Hispanic or Latino: - White alone"]/race["Estimate; Total:"]
race["Percent Black or African African alone"] = race["Estimate; Not Hispanic or Latino: - Black or African American alone"]/race["Estimate; Total:"]
race["Percent Asian alone"] = race["Estimate; Not Hispanic or Latino: - Asian alone"]/race["Estimate; Total:"]
race["Percent Hispanic"] = race["Estimate; Hispanic or Latino:"]/race["Estimate; Total:"]


edu = pd.read_csv(demog_data_path+ 'ACS_09_5YR_S1501_bach_clean.csv')
edu = edu.rename({"Percent of population 25 years and over  with Bachelor\'s degree": "Percent of population 25 years and over with Bachelor\'s degree"}, axis=1)
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].astype(float).dropna()


#get rid of + and ***
hvalue = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25077_hvalue_clean.csv')
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].apply(lambda x: x.replace("+", "").replace("-", "NaN").replace(",", ""))
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float)

#hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float).dropna()

#add attached and detached and find the %
single = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25024_singlefam_clean.csv')
single["Total Single Family"] = single["Estimate; 1, attached"]+ single["Estimate; 1, detached"]
single["% Single Family"] = single["Total Single Family"] / single["Estimate; Total:"]
#single["Tract"] = (single["Id2"].astype(str)).apply(lambda x: int(x[4:8]))

poverty = pd.read_csv(demog_data_path+"ACS_09_5YR_S1702_povertyfam_clean.csv")

# poverty = pd.read_csv(demog_data_path+ "ACS_17_5YR_S1701_poverty_clean.csv")
poverty['All families - Percent  below poverty level; Estimate; Families'] = poverty['All families - Percent  below poverty level; Estimate; Families'].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
poverty['All families - Percent  below poverty level; Estimate; Families'] = poverty['All families - Percent  below poverty level; Estimate; Families'].astype(float).dropna()



units = pd.read_csv(demog_data_path+'ACS_09_5YR_B25001_units_clean.csv')
#units["Tract"] = (units["Id2"].astype(str)).apply(lambda x: int(x[4:8]))


#find % renter occupied
tenure = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25003_tenure_clean.csv')
tenure["% Renter Occupied"] = tenure["Estimate; Renter occupied"]/tenure["Estimate; Total:"]
tenure["% Owner Occupied"] = tenure["Estimate; Owner occupied"]/tenure["Estimate; Total:"]


income = pd.read_csv(demog_data_path + 'ACS_09_5YR_S1903_income.csv')
income["Median income (dollars); All households"] = income["Median income (dollars); All households"].apply(remove_minus)

income["Median income (dollars); All households"] = income["Median income (dollars); All households"].astype(float)


#### Create Controlled features dataframe

In [17]:
race_units_merged = race.merge(units, how='left', left_on="Id2", right_on="Id2")
print("race and units merged", "length", len(race_units_merged))
#print("race and units merged", "dtypes", race_units_merged.dtypes)

edu_merged = race_units_merged.merge(edu, how='left', left_on="Id2", right_on="Id2")
print("edu merged", "length", len(edu_merged))
#print("edu merged", "dtypes", edu_merged.dtypes)

income_merged = edu_merged.merge(income, how='left', left_on="Id2", right_on="Id2")
print("income merged", "length", len(income_merged))
#print("income", "dtypes", income_merged.dtypes)

poverty_merged = income_merged.merge(poverty, how='left', left_on="Id2", right_on="Id2")
print("poverty merged", "length", len(poverty_merged))
#print("poverty", "dtypes", poverty_merged.dtypes)

hvalue_merged = poverty_merged.merge(hvalue, how='left', left_on="Id2", right_on="Id2")
print("hvalue merged", "length", len(hvalue_merged))
#print("hvalue", "dtypes", hvalue_merged.dtypes)

single_merged = hvalue_merged.merge(single, how='left', left_on="Id2", right_on="Id2")
print("single merged", "length", len(single_merged))
#print("singe merged", "dtypes", single_merged.dtypes)


merged = single_merged.merge(tenure, how='left', left_on="Id2", right_on="Id2")
print("tenure merged", "length", len(merged))


race and units merged length 7049
edu merged length 7049
income merged length 7049
poverty merged length 7049
hvalue merged length 7049
single merged length 7049
tenure merged length 7049


In [18]:
merged.columns
merged = merged[['Id2','Percent of population 25 years and over with Bachelor\'s degree', 
      'All families - Percent  below poverty level; Estimate; Families', 
      '% Single Family', '% Owner Occupied', 'Percent White alone',
       'Percent Black or African African alone', 'Percent Asian alone',
       'Percent Hispanic', 'Estimate; Total Number of Housing Units', 'Estimate; Median value (dollars)', 
        'Median income (dollars); All households']]
merged_rename = merged.rename({'All families - Percent  below poverty level; Estimate; Families': "% below poverty level",
              'Estimate; Total Number of Housing Units': 'Total number of housing units',
              'Estimate; Median value (dollars)': "Median home value",
              'Median income (dollars); All households': "Median income", 'Id2': "Geoid"}, axis=1)
#print("num tracts", len(merged))
      
merged_rename.head(50)





,Geoid,Percent of population 25 years and over with Bachelor's degree,% below poverty level,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Total number of housing units,Median home value,Median income
0,6001400100,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0
1,6001400200,37.6,0.0,0.665236,0.657428,0.767823,0.022640,0.069364,0.106936,932,909500.0,122647.0
2,6001400300,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0
3,6001400400,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0
4,6001400500,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0
5,6001400600,51.9,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,586700.0,39802.0
6,6001400700,19.4,8.1,0.572421,0.395335,0.356832,0.453986,0.095104,0.049218,2016,511600.0,32471.0
7,6001400800,33.5,21.7,0.423123,0.297976,0.326789,0.378702,0.174709,0.086190,1678,499100.0,37750.0
8,6001400900,28.2,4.0,0.492487,0.438356,0.376453,0.545182,0.007499,0.041245,1198,524600.0,51767.0
9,6001401000,19.0,31.7,0.466443,0.423432,0.166775,0.566320,0.143205,0.071196,2682,415600.0,36875.0


#### Get only Bay Area counties

In [19]:
county_codes = ['6001', '6013', '6041', '6055', '6075', '6081', '6085', '6097', '6095']


In [20]:
merged_bay = merged[merged["Id2"].apply(lambda x: str(x)[:4]).isin(county_codes)]
print("num tracts", len(merged_bay))
merged_bay.head()


num tracts 1405


,Id2,Percent of population 25 years and over with Bachelor's degree,All families - Percent below poverty level; Estimate; Families,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households
0,6001400100,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0
1,6001400200,37.6,0.0,0.665236,0.657428,0.767823,0.022640,0.069364,0.106936,932,909500.0,122647.0
2,6001400300,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0
3,6001400400,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0
4,6001400500,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0


#### Missing demog data exploration

In [21]:
print("number of tracts in the bay area:", len(merged_bay))
print("number of bay tracts with complete demographic info:", len(merged_bay.dropna())) 
print("percent of tracts that have at least one nan entry: ", (len(merged_bay)-len(merged_bay.dropna()))/len(merged_bay)) 

#by column
for col in merged_bay.columns[1:]:
    column = merged_bay[col]
    num_missing = len(column) - len(column.dropna())
    print(col, ", num missing:", num_missing)
    



number of tracts in the bay area: 1405
number of bay tracts with complete demographic info: 1382
percent of tracts that have at least one nan entry:  0.016370106761565837
Percent of population 25 years and over with Bachelor's degree , num missing: 2
All families - Percent  below poverty level; Estimate; Families , num missing: 4
% Single Family , num missing: 3
% Owner Occupied , num missing: 3
Percent White alone , num missing: 2
Percent Black or African African alone , num missing: 2
Percent Asian alone , num missing: 2
Percent Hispanic , num missing: 2
Estimate; Total Number of Housing Units , num missing: 0
Estimate; Median value (dollars) , num missing: 23
Median income (dollars); All households , num missing: 5


In [49]:
na_free = merged_bay.dropna()
only_na = merged_bay[~merged_bay.index.isin(na_free.index)]

#this is the dataframe of the census tracts that have at least one null value in their demographic variables, 
#merged with the ACS 2009 population estimates. There doesn't seem to be a connection between CTs with very 
#small populations and those with missing values, though the largest problem seems to be coming from the % poverty level
#column
pop_df = pd.read_csv(demog_data_path+"ACS_09_5YR_B01003_totalpop.csv")

na_ct_pop = pd.merge(only_na, pop_df, how='inner', right_on='Id2', left_on='Id2')
na_ct_pop

#### Merge Demographic features with neighbors dataframe to contribute CRA eligibility information by tract and exclude CTs without oppositely coded neighbors

Universe: all of the CTs in the 9 county bay area that had loan activity have oppositely coded neighbors. The boolean_tracts_all_2009.csv file includes all the tracts that have oppositely coded neighbors and their CRA eligibility.



In [23]:
data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/'
cra_elig_neighbors = pd.read_csv(data_path+ 'boolean_tracts_all_2009.csv')
cra_elig_neighbors.head()

,Tract,Year,CRA Eligible
0,6001433700,2009,eligible
1,6001435101,2009,not eligible
2,6001435101,2009,not eligible
3,6001435101,2009,not eligible
4,6001435101,2009,not eligible


In [24]:
demog_with_cra = pd.merge(merged, cra_elig_neighbors, how='right', left_on='Id2', right_on='Tract')
#.drop("Id2", axis=1) 
#i think that this is supposed to
#be a right merge because the demographics dataframe has all the census tracts, while the boolean neighbors one only
#has the census tracts in the bay area that have neighbors
demog_with_cra = demog_with_cra.drop(["Tract", "Year"], axis=1)
print(len(demog_with_cra))
demog_with_cra.head()



1228


,Id2,Percent of population 25 years and over with Bachelor's degree,All families - Percent below poverty level; Estimate; Families,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA Eligible
0,6001400100,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0,not eligible
1,6001400300,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0,not eligible
2,6001400400,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0,not eligible
3,6001400500,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0,eligible
4,6001400600,51.9,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,586700.0,39802.0,eligible


#### Get total number and percentages of loans data from HMDA files
This is currently for 2009, but next step is to do it for the average of the recovery period 2013-2017


Universe: all of the CTs in the 9 county bay area that had loan activity during 2009

In [25]:
filepath = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data/'
Alameda = pd.read_csv(filepath+'Alameda_2009_parsed.csv')
Alameda["Geoid"] = Alameda["Tract"].apply(lambda x: int(x*100 + 6001000000))

ContraCosta = pd.read_csv(filepath+'ContraCosta_2009_parsed.csv')
ContraCosta["Geoid"] = ContraCosta["Tract"].apply(lambda x: int(x*100 + 6013000000))

Napa = pd.read_csv(filepath+'Napa_2009_parsed.csv')
Napa["Geoid"] = Napa["Tract"].apply(lambda x: int(x*100 + 6055000000))

Marin = pd.read_csv(filepath+'Marin_2009_parsed.csv')
Marin["Geoid"] = Marin["Tract"].apply(lambda x: int(x*100 + 6041000000))

SanMateo = pd.read_csv(filepath+'SanMateo_2009_parsed.csv')
SanMateo["Geoid"] = SanMateo["Tract"].apply(lambda x: int(x*100 + 6081000000))

SanFrancisco = pd.read_csv(filepath+'SanFrancisco_2009_parsed.csv')
SanFrancisco["Geoid"] = SanFrancisco["Tract"].apply(lambda x: int(x*100 + 6075000000))

Solano = pd.read_csv(filepath+'Solano_2009_parsed.csv')
Solano["Geoid"] = Solano["Tract"].apply(lambda x: int(x*100 + 6095000000))

SantaClara = pd.read_csv(filepath+'SantaClara_2009_parsed.csv')
SantaClara["Geoid"] = SantaClara["Tract"].apply(lambda x: int(x*100 + 6085000000))

Sonoma = pd.read_csv(filepath+'Sonoma_2009_parsed.csv')
Sonoma["Geoid"] = Sonoma["Tract"].apply(lambda x: int(x*100 + 6097000000))

counties = [Alameda, ContraCosta, Napa, Marin, SanMateo, SanFrancisco,Solano,SantaClara, Sonoma]

bay_counties_loans = pd.concat(counties).reset_index(drop=True)
bay_counties_loans["Total Loans"] = bay_counties_loans['# HI borrower, LI tract'] + bay_counties_loans['# HI borrower, HI tract']+ bay_counties_loans['# LI borrower, LI tract']
+ bay_counties_loans['# LI borrower, HI tract']
bay_counties_loans = bay_counties_loans.drop(["Tract", "Year"], axis=1)


#### Merge Loans Data with cra_demog features

In [26]:
allvars_2009 = pd.merge(demog_with_cra, bay_counties_loans, how="left", left_on='Id2', right_on='Geoid').drop(['CRA Eligible_y', "Geoid"], axis=1)
allvars_2009['CRA'] = allvars_2009['CRA Eligible_x'].apply(lambda x: 1 if x == 'eligible' else 0)
allvars_2009.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/all_vars2009.csv")
print(allvars_2009.columns)
allvars_2009.head()

Index(['Id2', 'Percent of population 25 years and over with Bachelor's degree',
       'All families - Percent  below poverty level; Estimate; Families',
       '% Single Family', '% Owner Occupied', 'Percent White alone',
       'Percent Black or African African alone', 'Percent Asian alone',
       'Percent Hispanic', 'Estimate; Total Number of Housing Units',
       'Estimate; Median value (dollars)',
       'Median income (dollars); All households', 'CRA Eligible_x', 'County',
       'type', '% HI borrower, LI tract', '# HI borrower, LI tract',
       '% HI borrower, HI tract', '# HI borrower, HI tract',
       '% LI borrower, LI tract', '# LI borrower, LI tract',
       '%LI borrower, HI tract', '# LI borrower, HI tract', 'Total Loans',
       'CRA'],
      dtype='object')


,Id2,Percent of population 25 years and over with Bachelor's degree,All families - Percent below poverty level; Estimate; Families,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,...,"% HI borrower, LI tract","# HI borrower, LI tract","% HI borrower, HI tract","# HI borrower, HI tract","% LI borrower, LI tract","# LI borrower, LI tract","%LI borrower, HI tract","# LI borrower, HI tract",Total Loans,CRA
0,6001400100,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,...,0.000000,0.0,0.981818,54.0,0.000000,0.0,0.018182,1.0,54.0,0
1,6001400300,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,...,0.000000,0.0,0.906250,58.0,0.000000,0.0,0.093750,6.0,58.0,0
2,6001400400,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,...,0.000000,0.0,0.979167,47.0,0.000000,0.0,0.020833,1.0,47.0,0
3,6001400500,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,...,0.877193,50.0,0.000000,0.0,0.122807,7.0,0.000000,0.0,57.0,1
4,6001400600,51.9,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,...,0.870968,27.0,0.000000,0.0,0.129032,4.0,0.000000,0.0,31.0,1


#### Regression Data Prep

In [27]:
allvars_2009.columns
noinc = ['% HI borrower, LI tract',
       '# HI borrower, LI tract', '% HI borrower, HI tract',
       '# HI borrower, HI tract', '% LI borrower, LI tract',
       '# LI borrower, LI tract', '%LI borrower, HI tract',
       '# LI borrower, HI tract', 'Geoid', 'Total Loans', "CRA Eligible_x", "type", "County", "Id2"]

X_vars = allvars_2009.loc[:, ~allvars_2009.columns.isin(noinc)]
y = allvars_2009["Total Loans"]
print(len(y))
print(len(X_vars))
X_vars.head()

1228
1228


,Percent of population 25 years and over with Bachelor's degree,All families - Percent below poverty level; Estimate; Families,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0,0
1,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0,0
2,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0,0
3,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0,1
4,51.9,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,586700.0,39802.0,1


In [61]:
# This is the number of nans (missing data) 
# for col in X_vars.columns:
#     num = sum(np.isnan(X_vars[col]))
#     print(col, num)
    
    

#### Some of the data is Nan, I dealt with this is 3 ways, discuss the best strategy for this with Carolina

Strategy 1: drop all the Nans, lose ~20% of the data

In [37]:
#need to concatenate x with y, dropna and then unconcatenate in order to do this correctly
xydf = X_vars.copy()
xydf["y"] = y
dropped = xydf.dropna()

X_drop = dropped.loc[:, dropped.columns != 'y']
y_drop = dropped['y']

print("num tracts", len(X_drop))
print("y", len(y_drop))
y_drop.head()

num tracts 1210
y 1210


0    54.0
1    58.0
2    47.0
3    57.0
4    31.0
Name: y, dtype: float64

Strategy 2: infill nans with mean of column

In [29]:
X_vars_mean = X_vars.fillna(X_vars.mean())
y_mean = y.fillna(y.mean())
print("num tracts", len(X_vars))
print("num y", len(y))
X_vars_mean.head()


num tracts 1228
num y 1228


,Percent of population 25 years and over with Bachelor's degree,All families - Percent below poverty level; Estimate; Families,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0,0
1,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0,0
2,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0,0
3,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0,1
4,51.9,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,586700.0,39802.0,1


Strategy 3: infill nans with median of column

In [30]:
X_vars_med = X_vars.fillna(X_vars.median())

y_med = y.fillna(y.median())
print("X", len(X_vars_med))
print("y", len(y_med))

X_vars_med.head()


X 1228
y 1228


,Percent of population 25 years and over with Bachelor's degree,All families - Percent below poverty level; Estimate; Families,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,34.5,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,1000000.0,186439.0,0
1,32.1,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,718100.0,66638.0,0
2,44.0,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,790500.0,80391.0,0
3,28.1,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,572000.0,50658.0,1
4,51.9,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,586700.0,39802.0,1


#### Data Normalization

In [31]:
def normalize_columns(data, mean_df, std_df):
    '''
    Input:
      data (data frame): contains only numeric columns
    Output:
      data frame, the same data, except each column is standardized 
      to have 0-mean and unit variance
    '''
    normalized_data=(data-mean_df.mean())/std_df.std()

    return normalized_data

In [32]:
def unnormalize_columns(data, mean_df, std_df):
    unnormalized_data=(data*std_df.std())+mean_df.mean()

    return unnormalized_data

#### Regression Using dropped NaNs method

In [33]:
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [34]:
#normalize the data before perfoming the regression
normal_x_drop = normalize_columns(X_drop, X_drop, X_drop)
normal_y_drop = normalize_columns(y_drop, y_drop, y_drop)

In [35]:
from sklearn import linear_model
sk_model = linear_model.LinearRegression()
sk_model.fit(normal_x_drop,normal_y_drop)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [36]:
X_drop_constant = sm.add_constant(normal_x_drop)
sm_model = sm.regression.linear_model.OLS(normal_y_drop, X_drop_constant)
results = sm_model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     167.1
Date:                Wed, 16 Jan 2019   Prob (F-statistic):          1.09e-245
Time:                        14:35:31   Log-Likelihood:                -1121.1
No. Observations:                1210   AIC:                             2268.
Df Residuals:                    1197   BIC:                             2334.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [48]:
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/prelim_regression_results.html', 'w') as outfile:
    outfile.write(results.summary().as_html())

#### This is the scikitlearn output, which is redundant with the statsmodels output above.

In [71]:
# from collections import OrderedDict
# linear_coef_dict = {}
# for linear_coef, linear_feature in zip(sk_model.coef_, X_vars.columns):
#     linear_coef_dict[linear_feature] = linear_coef
    
# sorted_coefs_linear = OrderedDict(sorted(linear_coef_dict.items(), key=lambda t: t[1]))
# sorted_coefs_linear


#### Regression using filled means method

In [72]:
normal_x_vars_mean = normalize_columns(X_vars_mean, X_vars_mean, X_vars_mean)
normal_y_mean = normalize_columns(y_mean, y_mean, y_mean)

In [73]:
from sklearn import linear_model
sk_model_mean = linear_model.LinearRegression()
sk_model_mean.fit(normal_x_vars_mean,normal_y_mean)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [74]:
X_mean_constant = sm.add_constant(normal_x_vars_mean)
sm_model = sm.regression.linear_model.OLS(normal_y_mean, X_mean_constant)
results = sm_model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            Total Loans   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     161.4
Date:                Tue, 15 Jan 2019   Prob (F-statistic):          1.89e-241
Time:                        23:35:47   Log-Likelihood:                -1156.6
No. Observations:                1228   AIC:                             2339.
Df Residuals:                    1215   BIC:                             2406.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [47]:
# with open('regression_results.html', 'w') as outfile:
#     outfile.write(results.summary().as_html())

In [75]:
# from collections import OrderedDict
# linear_coef_dict = {}
# for linear_coef, linear_feature in zip(sk_model_mean.coef_, X_vars.columns):
#     linear_coef_dict[linear_feature] = linear_coef
    
# sorted_coefs_linear_mean = OrderedDict(sorted(linear_coef_dict.items(), key=lambda t: t[1]))
# sorted_coefs_linear_mean

#### Regression using filled medians method


In [76]:
normal_x_median = normalize_columns(X_vars_med, X_vars_med, X_vars_med)
normal_y_median = normalize_columns(y_med, y_med, y_med)
normal_x_median.head()

,Percent of population 25 years and over with Bachelor's degree,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent White alone,Percent Black or African African alone,Percent Asian alone,Percent Hispanic,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,1.289540,-1.024133,1.089326,1.527403,1.516134,-0.360677,-0.507372,-1.401598,-0.538142,2.274438,4.727079,-0.753568
1,1.048455,-0.559209,-0.543388,-0.720945,1.274925,0.143072,-0.719931,-0.964692,0.671644,0.761496,-0.131244,-0.753568
2,2.243834,-0.319702,-0.254675,-0.589601,1.204503,0.086081,-0.704224,-1.089341,-0.022073,1.150063,0.426485,-0.753568
3,0.646647,-0.291525,-0.672811,-0.571645,-0.016946,2.304835,-0.776139,-0.792095,-0.275222,-0.022614,-0.779286,1.325940
4,3.037404,-0.080195,0.035994,-0.420097,-0.373095,3.409778,-0.981697,-1.107975,-1.119940,0.056280,-1.219532,1.325940


In [77]:
from sklearn import linear_model
sk_model_med = linear_model.LinearRegression()
sk_model_med.fit(normal_x_median,normal_y_median)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [78]:
normal_x_median_constant = sm.add_constant(normal_x_median)
sm_model = sm.regression.linear_model.OLS(normal_y_median, normal_x_median_constant)
results = sm_model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            Total Loans   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     163.7
Date:                Tue, 15 Jan 2019   Prob (F-statistic):          9.78e-244
Time:                        23:35:47   Log-Likelihood:                -1151.3
No. Observations:                1228   AIC:                             2329.
Df Residuals:                    1215   BIC:                             2395.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [79]:
# from collections import OrderedDict
# linear_coef_dict = {}
# for linear_coef, linear_feature in zip(sk_model_med.coef_, X_vars.columns):
#     linear_coef_dict[linear_feature] = linear_coef
    
# sorted_coefs_linear_med = OrderedDict(sorted(linear_coef_dict.items(), key=lambda t: t[1]))
# sorted_coefs_linear_med